In [1]:
import matplotlib.pyplot as plt
import wfdb
from tensorly.decomposition import partial_tucker
from sklearn.model_selection import train_test_split
from scipy.signal import detrend
import numpy as np
from ecgdetectors import Detectors
from tqdm import tqdm
import joblib

Using numpy backend.


In [ ]:
class_dict = {'Myocardial infarction':'MI', 
             'Healthy control':'CON', 
             'Cardiomyopathy':'C/HF', 
             'Bundle branch block':'BBB', 
             'Dysrhythmia':'D',
             'Hypertrophy':'MHY',
             'Valvular heart disease':'VHD',
             'Myocarditis':'MYC',
             'Heart failure (NYHA 3)':'C/HF',
             'Heart failure (NYHA 2)':'C/HF',
             'Heart failure (NYHA 4)':'C/HF'}

MICON_dict = {'Myocardial infarction':'MI', 
             'Healthy control':'CON'}

In [3]:
def patient_classes(folder):
    '''Creates a unique list of patients with their respective classes, droping unlabeled data.
        folder: the folder path containing the data eg ptb-diagnostic-ecg-database-1.0.0/'''
    patient_class = []
    with open(folder+'RECORDS') as f:
        file_names = [x.strip() for x in f]
    for x in file_names:
        patient = x.split('/')[0]
        with open(folder+x+'.hea') as f:
            for line in f:
                if 'Reason for admission:' in line:
                    try:
                        patient_class.append((patient, class_dict[line.split(':')[-1].strip()]))
                    except:
                        pass
    return [*{*patient_class}]

In [2]:
def patient_tt_split(zip_list, train, test, classes=None):
    '''Splits the patients into a train and test set before processing to prevent patient level data leakage to the test set.
        zip_list: a list of (patient_id, class) tuples
        train: training proportion
        test: testing proportion
        classes: optional (default: None) A list of classes to include in the data sets, None includes all classes'''
    if classes == None:
        X, y = zip(*zip_list)
        return train_test_split(X, y, train_size=train, test_size=test, stratify=y, random_state=1)
    else:
        X, y = zip(*[x for x in zip_list if x[1] in classes])
        return train_test_split(X, y, train_size=train, test_size=test, stratify=y, random_state=1)

In [5]:
def data_load_ptb(folder, patient_set):
    '''Extracts the data from the files, processes it and returns an array and target list.
        folder: data folder path
        patient set: a list of patient file names to process'''
    full_data = []
    target = []
    global error_list
    error_list = []
    with open(folder+'RECORDS') as f:
        file_names = [x.strip() for x in f]
    for file in tqdm(file_names):
        for patient in patient_set:
            if patient in file:
                raw = wfdb.rdsamp(folder+file)
                class_label = raw[1]['comments'][4].split(': ')[-1]
                try:
                    target.append(MICON_dict[class_label])
                    full_data.append(wave_processing_pad(raw[0]))
                except:
                    pass
    return np.array(full_data), target

In [8]:
def wave_processing_pad(data):
    '''R peaks are detected and compared between channels 0 and 1 for redundancy. Signals with peaks taller half the R peak height in the ST segment are discarded.
    Beats are sliced by R index and placed in an array padded with zeros to 200 beats as some recordings contain less beats.'''
    red_channel_data = data[:,0:12]
    
    detector = Detectors(1000)
    swt_1_peaks = detector.swt_detector(detrend(red_channel_data[:,1]))[1:-1]
    swt_0_peaks = detector.swt_detector(detrend(red_channel_data[:,0]))[1:-1]
    
    if len(swt_1_peaks) > len(swt_0_peaks):
        small_list = swt_0_peaks
        long_list = swt_1_peaks
    else:
        small_list = swt_1_peaks
        long_list = swt_0_peaks
    
    diff = [swt_0_peaks[i]-swt_1_peaks[i] for i in range(len(small_list))]

    def filt_sig_proc():
        try:
            for x in range(n_channels):
                if x == 0:
                    sig = detrend(red_channel_data[slice(y-250, y+350),x])
                    if max(sig)/2 < max(sig[300:420]):
                        break
                    else:
                        ecg_vec[i,:,x] = sig
                else:
                    ecg_vec[i,:,x] = detrend(red_channel_data[slice(y-250, y+350),x])
        except:
            pass

    
    
    n_beats = 200
    n_channels = 12
    n_samples = 600
    ecg_vec = np.zeros((n_beats, n_samples, n_channels))
    for i,y in enumerate(small_list):
        if abs(diff[i]) > 20:
            for z in long_list:
                if abs(y-z) > 50:
                    pass
                else:
                    filt_sig_proc()
        else:
            filt_sig_proc()
    return ecg_vec

In [9]:
def stack_and_tensor(data):
    return tensorly.base.vec_to_tensor(np.stack(data), (len(data), *data[0].shape))

In [10]:
def train_test_tensor(X, y, train_size, test_size, random_state):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, test_size=test_size, random_state=random_state, stratify=y)
    return stack_and_tensor(X_train), stack_and_tensor(X_test), np.array(y_train), np.array(y_test)

In [11]:
def decomposition_fit(tensor, n_channels):
    return partial_tucker(tensor, (0,1,3), ranks=(1, 1, 600, n_channels), init='random', n_iter_max=10000)

In [12]:
def reshapeing(X, y, channels):
    '''Reshapes data removing the recording axis and any padding beats, as well as some channels if dimensionality reduction has been applied.
    X: data (array)
    y: data labels (list or array)
    channels: number of channels (int)'''
    mask = np.zeros((X.shape[0], X.shape[1]))
    new_y = []
    for record in range(X.shape[0]):
        for beat in range(X.shape[1]):
            if np.any(X[record,beat,:,:]):
                mask[record,beat] = 1
                new_y.append(y[record])
    
    print(len(new_y))
    print(mask.sum())
    new_X = np.zeros((len(new_y), X.shape[2], channels))

    count = 0
    for record in range(X.shape[0]):
        for beat in range(X.shape[1]):
            if mask[record,beat]:
                new_X[count,:,:channels] = X[record,beat,:,:channels]
                count += 1
    return new_X, new_y

# MICON Stuff

In [23]:
patient_zip = patient_classes('/Users/foleyc/ga/DSI11-lessons/projects/project-capstone/resources/ptb-diagnostic-ecg-database-1.0.0/')

In [25]:
train_patients, test_patients, ytr , ytest = patient_tt_split(patient_zip, 0.7, 0.3, classes=['MI', 'CON'])

In [28]:
X_train, y_train = data_load_ptb('/Users/foleyc/ga/DSI11-lessons/projects/project-capstone/resources/ptb-diagnostic-ecg-database-1.0.0/', 'RECORDS', train_patients)

100%|██████████| 549/549 [03:15<00:00,  2.81it/s]


In [18]:
X_test, y_test = data_load_ptb('C://Users/Calum/project_work/ptb-diagnostic-ecg-database-1.0.0/', 'RECORDS', test_patients)

100%|████████████████████████████████████████| 549/549 [02:08<00:00,  4.26it/s]


In [149]:
X_train_full, y_train_full = reshapeing(X_train, y_train, 12)

37077
37077.0


In [20]:
X_test_full, y_test_full = reshapeing(X_test, y_test, 12)

17583
17583.0


In [23]:
joblib.dump(X_train_full, 'full/X_train')

joblib.dump(X_test_full, 'full/X_test')

joblib.dump(y_train_full, 'full/y_train')

joblib.dump(y_test_full, 'full/y_test')

['full/y_test']

# Tensor Decomposition

In [148]:
decomp = decomposition_fit(X_train, 6)

In [150]:
for i in range(X_train.shape[0]):
    for j in range(X_train.shape[1]):
        X_train[i,j,:,:] = np.dot(X_train[i,j,:,:], decomp[1][2])

In [25]:
for i in range(X_test.shape[0]):
    for j in range(X_test.shape[1]):
        X_test[i,j,:,:] = np.dot(X_test[i,j,:,:], decomp[1][2])

#  

In [151]:
X_train_red, y_train_red = reshapeing(X_train, y_train, 6)

37077
37077.0


In [27]:
X_test_red, y_test_red = reshapeing(X_test, y_test, 6)

17583
17583.0


In [153]:
X_train_red.shape, X_train_full.shape

((37077, 600, 6), (37077, 600, 12))

In [28]:
joblib.dump(X_train_red, 'reduced/X_train')

joblib.dump(X_test_red, 'reduced/X_test')

joblib.dump(y_train_red, 'reduced/y_train')

joblib.dump(y_test_red, 'reduced/y_test')

['reduced/y_test']